If you are interested in graident boosting, here is a good place to start: https://xgboost.readthedocs.io/en/latest/tutorials/model.html

This is a supervised machine learning method.

XGB does not like to have multiple outputs: https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html

In [5]:
!pip install xgboost --upgrade

Requirement already up-to-date: xgboost in /usr/local/lib/python3.6/dist-packages (1.3.3)


In [34]:
# If you have installation questions, please reach out

import pandas as pd # data storage
import xgboost      # graident boosting 
import numpy as np  # math and stuff

import xgboost as xgb
import datetime
print('xgb version:',xgb.__version__)

import sklearn  
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, max_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt # plotting utility

xgb version: 1.3.3


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset_inner_imputed_12_21_2020.csv')

df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df.describe()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.468406,0.325999,0.430844,0.811259,0.613371,0.351642,1.011018,0.172737,0.172737,0.351702,0.326029,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.191316,0.237404,0.263514,0.162843,0.214604,0.186912,1.503608,0.226664,0.226664,0.178735,0.229901,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.000000,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.360072,0.122318,0.174146,0.743878,0.424460,0.216157,-0.037336,0.004595,0.004595,0.220800,0.131058,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.275825,0.471657,0.867010,0.605516,0.329327,0.795008,0.005532,0.005532,0.337049,0.283192,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530980,0.662807,0.925012,0.830179,0.470706,1.378387,0.332503,0.332503,0.463317,0.525448,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

## Loading in Entire Dataset

In [9]:
df.columns.values

array(['depth_ft', 'USGS_ID', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 'PE_smooth', 'GR_smooth',
       'gz_linear_interp', 'gz_pchip_interp', 'perm_air_md',
       'perm_klink_md', 'He_por', 'horz_perm_md', 'effective_por',
       'por_percent', 'stress_swanson_perm_md', 'sample', 'Na', 'Na Err',
       'Mg', 'Mg Err', 'Al', 'Al Err', 'Si', 'Si Err', 'P', 'P Err', 'S',
       'S Err', 'K', 'K Err', 'Ca', 'Ca Err', 'Ti', 'Ti Err', 'Cr',
       'Cr Err', 'Mn', 'Mn Err', 'Fe', 'Fe Err', 'Co', 'Co Err', 'Ni',
       'Ni Err', 'Cu', 'Cu Err', 'Zn', 'Zn Err', 'Ba', 'Ba Err'],
      dtype=object)

In [10]:
dataset = df[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'Ti', 'Mg', 'Si', 'Al', 'Ca'
]]

In the next code block, we will remove the rows without data, and change string NaN's to np.nans

In [11]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 
dataset = dataset.dropna()
np.shape(dataset)

dataset.head(3)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,PE_smooth,Ti,Mg,Si,Al,Ca
0,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.183350,0.366294,0.1641,0.6987,33.9921,2.8101,4.5640
5,0.626112,0.182393,0.063209,0.586778,0.825487,0.351442,0.922670,0.001810,0.001810,0.196369,0.357995,0.1746,0.9547,33.9915,2.9962,3.8975
10,0.621729,0.184446,0.063426,0.590484,0.825878,0.402086,0.886733,0.001889,0.001889,0.211611,0.360458,0.1555,0.7553,33.3257,2.5388,4.1047


In [12]:
X = dataset[[ 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

Y2 = dataset[['Ti', 'Mg', 'Si', 'Al', 'Ca']]


In [13]:
#scaler = MinMaxScaler()
scaler = RobustScaler()

#Y = scaler.fit_transform(Y2[Y2.columns].to_numpy())
df_Y = pd.DataFrame(data=Y, columns=Y2.columns)

In [14]:
df_Y.describe()

,Ti,Mg,Si,Al,Ca
count,2396.000000,2396.000000,2396.000000,2.396000e+03,2.396000e+03
mean,0.222155,0.078920,0.132472,1.683226e-01,6.729072e-02
std,0.646981,0.543654,0.601385,6.673984e-01,7.386606e-01
min,-0.821153,-0.717823,-2.855834,-1.009715e+00,-8.881006e-01
25%,-0.282225,-0.443103,-0.263516,-3.405999e-01,-5.380505e-01
50%,0.000000,0.000000,0.000000,-1.047068e-16,-8.016997e-17
75%,0.717775,0.556897,0.736484,6.594001e-01,4.619495e-01
max,3.535806,2.516721,1.378160,3.317291e+00,4.073023e+00


## Starting to set up the ML model params

In [27]:
seed = 7 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, df_Y.values, test_size=test_size)

### Let's try some hyperparameter tuning (this takes forever!)

Hyperparameter testing does a grid search to find the best parameters, out of the parameters below. This turned out to be really slow on my laptop. Please skip this! 

In [28]:
multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective ='reg:squarederror', 
              colsample_bytree = 0.9, 
              learning_rate = 0.1, 
              max_depth = 5, 
              n_estimators = 60))

multioutputregressor.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/data.py:107: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.9, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.1,
                                            max_delta_step=None, max_depth=5,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=60, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='reg:squarederror',
                                            rando

In [29]:
preds = multioutputregressor.predict(X_test)

In [30]:
rmse = mean_squared_error(scaler.inverse_transform(y_test), scaler.inverse_transform(preds), squared=False)
print("Mean Squared Error: %f" % (rmse))

Mean Squared Error: 1.419285


In [35]:
rmse2 = mean_squared_error(y_test, preds, squared=False)
print("Mean Squared Error: %f" % (rmse2))

MAE = median_absolute_error(y_test, preds)
print("Median Abs Error: %f" % (MAE))

Mean Squared Error: 0.400975
Median Abs Error: 0.183426


# Exporting Results

In [36]:
x = datetime.datetime.now()

d = {'target': ['MultiXRF'],
     'MSE': [rmse2], 
     'MAE': [MAE],
     'day': [x.day], 
     'month':[x.month], 
     'year':[x.year],
     'model':['XGB'],
     'version':[xgb.__version__ ]}

results = pd.DataFrame(data=d)

results.to_csv('drive/My Drive/1_lewis_research/analysis/experiments/xgb/results_xgb/multi_xrf_xgb.csv')

results

,target,MSE,MAE,day,month,year,model,version
0,MultiXRF,0.400975,0.183426,15,2,2021,XGB,1.3.3


In [20]:
true_preds = scaler.inverse_transform(preds)
true_preds_df = pd.DataFrame(data = true_preds,   
                  columns = ['Ti', 'Mg', 'Si', 'Al', 'Ca'])
 
y_test_trans = scaler.inverse_transform(y_test)
y_test_df = pd.DataFrame(data = y_test_trans,   
                  columns = ['Ti', 'Mg', 'Si', 'Al', 'Ca'])

In [21]:
true_preds_df.describe(percentiles=[0.05, 0.95])

,Ti,Mg,Si,Al,Ca
count,599.000000,599.000000,599.000000,599.000000,599.000000
mean,0.186205,1.384597,301.177124,10.524016,10.432483
std,0.008364,0.346049,32.781494,2.031411,4.236076
min,0.171218,0.826423,212.241943,6.272173,4.044814
5%,0.176312,0.876121,263.111377,7.682854,4.647402
50%,0.184192,1.383846,285.607666,10.206944,10.663998
95%,0.203896,1.898356,354.604160,14.167496,16.498532
max,0.209304,1.984581,382.782288,17.241516,28.807932


In [22]:
yoyo = pd.DataFrame(data = preds,   
                  columns = ['Ti', 'Mg', 'Si', 'Al', 'Ca'])

yoyo.describe(percentiles=[0.05, 0.95])

,Ti,Mg,Si,Al,Ca
count,599.000000,599.000000,599.000000,599.000000,599.000000
mean,0.190265,0.770793,32.775459,3.485470,2.830383
std,0.061436,0.385120,3.989947,0.958010,1.529449
min,0.080194,0.149600,21.950865,1.480310,0.524092
5%,0.117602,0.204910,28.142352,2.145584,0.741658
50%,0.175485,0.769958,30.880451,3.335940,2.913969
95%,0.320205,1.342558,39.278247,5.203728,5.020546
max,0.359927,1.438519,42.707901,6.653430,9.464894


In [23]:
y_test_df.describe(percentiles=[0.05, 0.95])

,Ti,Mg,Si,Al,Ca
count,599.000000,599.000000,599.000000,599.000000,599.000000
mean,0.186411,1.392016,300.573865,10.449298,10.678167
std,0.011915,0.434298,39.819612,2.942931,5.677160
min,0.167393,0.734232,161.800779,5.557560,3.062433
5%,0.172327,0.813286,240.200801,6.608456,3.691260
50%,0.182261,1.350008,292.119260,9.717883,10.482392
95%,0.207326,2.063807,365.125379,15.761399,20.770849
max,0.236762,3.215128,386.957658,24.692713,39.814081


In [24]:
df80 = pd.DataFrame(data = y_test,   
                  columns = ['Ti', 'Mg', 'Si', 'Al', 'Ca'])

In [25]:
df80.describe()

,Ti,Mg,Si,Al,Ca
count,599.000000,599.000000,599.000000,599.000000,599.000000
mean,0.191781,0.779051,32.702038,3.450234,2.919085
std,0.087513,0.483332,4.846579,1.387880,2.049757
min,0.052100,0.047000,15.811500,1.143300,0.169400
25%,0.124800,0.320900,29.667700,2.367100,1.228950
50%,0.161300,0.732300,31.673000,3.105300,2.848400
75%,0.257850,1.186700,36.441950,4.471200,4.055350
max,0.561600,2.808000,43.216100,10.167400,13.438700
